In [2]:
import copy
import time
import h5py
import logging
import argparse
import sys, os
import random
import cupy as cp
import numpy.random as npr
import numpy  as np
import pde 
from math import pi
import numpy.linalg as LA
import scipy.linalg as spla
import matplotlib as mpl
# mpl.use('Qt5Agg')
import matplotlib.pyplot as plt
from datetime import datetime

from os.path import abspath, join, dirname, expanduser
from LevelSetPy.Grids import *
from LevelSetPy.Utilities import *
from LevelSetPy.Visualization import *
from LevelSetPy.DynamicalSystems import *
from LevelSetPy.BoundaryCondition import *
from LevelSetPy.InitialConditions import *
from LevelSetPy.SpatialDerivative import *
from LevelSetPy.ExplicitIntegration import *
from LevelSetPy.Visualization import RCBRTVisualizer


# the resolution was just taking things out the folder -- likely has something to do with the way things were pip installed via setup idk


In [3]:
def get_avoid_brt(flock, compute_mesh=True, color='crimson'):
	"""
		Get the avoid BRT for this flock. That is, every bird
		within a flock must avoid one another.

		Parameters:
		==========
		.flock: This flock of vehicles.
		.compute_mesh: compute mesh of local payoffs for each bird in this flock?
	"""
	for vehicle in flock.vehicles:
		# make the radius of the target setthe turn radius of this vehicle
		vehicle.payoff = shapeCylinder(flock.grid, 2, center=flock.update_state(vehicle.cur_state), \
										radius=vehicle.payoff_width)
	"""
		Now compute the overall payoff for the flock
	   	by taking a union of all the avoid sets.
	"""
	flock.payoff = shapeUnion([veh.payoff for veh in flock.vehicles])
	if compute_mesh:
		spacing=tuple(flock.grid.dx.flatten().tolist())
		flock.mesh_bundle = implicit_mesh(flock.payoff, 0, spacing,edge_color=None, face_color=color)

	return flock

def get_flock(gmin, gmax, num_points, num_agents, init_xyzs, label,\
				periodic_dims=2, reach_rad=.2, avoid_rad=.3,
				base_path='', save=True, color='blue'):
	"""
		Params
		======
		gmin: minimum bounds of the grid
		gmax: maximum bounds of the grid
		num_points: number of points on the grid
		num_agents: number of agents in this flock
		init_xyzs: initial positions of the individuals in this flock
		label: topological label of this flock among all flocks
		periodic_dims: periodic dimensions (usually theta: see)
		reach_rad: reach for external disturbance
		avoid_rad: avoid radius for topological interactions
	"""
	global u_bound, w_bound

	assert gmin.ndim==2, 'gmin must be of at least 2 dims'
	assert gmax.ndim==2, 'gmax must be of at least 2 dims'

	gmin = to_column_mat(gmin)
	gmax = to_column_mat(gmax)

	grid = createGrid(gmin, gmax, num_points, periodic_dims)

	vehicles = [Bird(grid, u_bound, w_bound, np.expand_dims(init_xyzs[i], 1), random.random(), \
					center=np.zeros((3,1)), neigh_rad=3, label=i+1, init_random=False) \
					for i in range(num_agents)]
	flock = Flock(grid, vehicles, label=label, reach_rad=.2, avoid_rad=.3)
	get_avoid_brt(flock, compute_mesh=True, color=color)

	if visualize and flock_payoff:
		visualize_init_avoid_tube(flock, save, fname=join(base_path, f"flock_{flock.label}.jpg"))
		plt.show()

	return flock


In [4]:
gMin = np.array([[-10, -10, -np.pi]]).T
gMax = np.array([[10, 10, np.pi]]).T
numPoints = 150
pDim = 2

u_bound = 1
w_bound = 1 

numAgents = 6

mean1 = np.array([-2.5, -2.5, np.pi / 2])
mean2 = np.array([7.5, 7.5, np.pi / 2])

flock_num=2
silent=True
save=True
out_dir='./data'
visualize=True
flock_payoff=False
resume=None
load_brt=False
verify=False
elevation=25.0
azimuth=45.0
pause_time=0.3
verbose=False

In [5]:
color = iter(plt.cm.inferno_r(np.linspace(.25, 1, numAgents)))

cov_mat = np.diag([.7, .7, .2])
initPosOne = npr.multivariate_normal(mean1, cov_mat, size=(1,6)).reshape(6,3)
initPosTwo = npr.multivariate_normal(mean2, cov_mat, size=(1,6)).reshape(6,3)

base_path1 = join("/opt/LevPy/Murmurations", 
                f"flock_{1}",
                datetime.strftime(datetime.now(), '%m-%d-%y_%H-%M'))

base_path2 = join("/opt/LevPy/Murmurations", 
                f"flock_{2}",
                datetime.strftime(datetime.now(), '%m-%d-%y_%H-%M'))


flock1 = get_flock(gMin, gMax, numPoints, numAgents, initPosOne, label=numAgents,
                    periodic_dims=pDim, reach_rad=.2, avoid_rad=.3, base_path=base_path1, 
                    color=next(color))

flock2 = get_flock(gMin, gMax, numPoints, numAgents, initPosTwo, label=numAgents,
                    periodic_dims=pDim, reach_rad=.2, avoid_rad=.3, base_path=base_path2, 
                    color=next(color))

finite_diff_data1 = Bundle(dict(innerFunc = termLaxFriedrichs,
            innerData = Bundle({'grid':flock1.grid,
                'hamFunc': flock1.hamiltonian,
                'partialFunc': flock1.dissipation,
                'dissFunc': artificialDissipationGLF,
                'CoStateCalc': upwindFirstWENO5a, 
                }),
                positive = True,  # direction to grow the updated level set
            ))

finite_diff_data2 = Bundle(dict(innerFunc = termLaxFriedrichs,
            innerData = Bundle({'grid':flock2.grid,
                'hamFunc': flock2.hamiltonian,
                'partialFunc': flock2.dissipation,
                'dissFunc': artificialDissipationGLF,
                'CoStateCalc': upwindFirstWENO5a, 
                }),
                positive = True,  # direction to grow the updated level set
            ))
pause_time=0.3

params1 = Bundle(
                {"grid": flock1.grid,
                'disp': True,
                'labelsize': 16,
                'labels': "Initial 0-LevelSet",
                'linewidth': 2,
                'elevation': 10,
                'azimuth': 5,
                'mesh': flock1.mesh_bundle,
                'pause_time': pause_time,
                'title': f'Initial BRT. Flock with {flock1.N} agents.',
                'level': 0, # which level set to visualize
                'winsize': (16,9),
                'fontdict': {'fontsize':18, 'fontweight':'bold'},
                "savedict": Bundle({"save": True,
                                "savename": "murmur",
                                "savepath": join("/opt/LevPy/Murmurations")
                })
        })

params2 = Bundle(
                {"grid": flock2.grid,
                'disp': True,
                'labelsize': 16,
                'labels': "Initial 0-LevelSet",
                'linewidth': 2,
                'elevation': 10,
                'azimuth': 5,
                'mesh': flock2.mesh_bundle,
                'pause_time': pause_time,
                'title': f'Initial BRT. Flock with {flock2.N} agents.',
                'level': 0, # which level set to visualize
                'winsize': (16,9),
                'fontdict': {'fontsize':18, 'fontweight':'bold'},
                "savedict": Bundle({"save": True,
                                "savename": "murmur",
                                "savepath": join("/opt/LevPy/Murmurations")
                })
        })



In [6]:
def evolve_brats_in_time(argFlockNum, argVisualize, flock, finite_diff_data, params):
	logging.getLogger('matplotlib.font_manager').disabled = True
	logger = logging.getLogger(__name__)

	times, norm_tracker = [], []
	
	num_agents = len(flock.vehicles)
	
	os.makedirs(params.savedict.savepath) if not os.path.exists(params.savedict.savepath) else None
	
	if argVisualize:
		viz = RCBRTVisualizer(params=params)
	
	t_range = [0, 100]
	t_steps = (t_range[1] - t_range[0]) / 10000
	small = 100*eps

	# Loop through t_range (subject to a little roundoff).
	t_now = t_range[0]
	start_time = time.perf_counter()
	itr_start = cp.cuda.Event()
	itr_end = cp.cuda.Event()

	value_rolling = cp.asarray(copy.copy(flock.payoff))

	colors = iter(plt.cm.ocean(np.linspace(.25, 2, 100)))
	color = next(colors)
	options = Bundle(dict(factorCFL=0.7, stats='on', singleStep='on'))

	# murmur flock savename
	
	savename = join(params.savedict.savepath, rf"murmurations_flock_{argFlockNum}_{datetime.strftime(datetime.now(), '%m-%d-%y_%H-%M')}.hdf5")
	if os.path.exists(savename):
		os.remove(savename)

	spacing = flock.grid.dx.flatten()
	
	gpu_time_buffer = []
	itr_start = cp.cuda.Event(); itr_end = cp.cuda.Event()

	while(t_range[1] - t_now > small * t_range[1]):
		itr_start.record()
		time_step = f"{t_now:.2f}/{t_range[-1]}"

		# Reshape data array into column vector for ode solver call.
		y0 = value_rolling.flatten()

		# How far to step?
		t_span = np.hstack([ t_now, min(t_range[1], t_now + t_steps) ])
		
		# Integrate a timestep.
		t, y, _ = odeCFL3(termRestrictUpdate, t_span, y0, odeCFLset(options), finite_diff_data)
		t_now = t

		# Get back the correctly shaped data array
		value_rolling = y.reshape(flock.grid.shape)

		# track the evolution of the growth of the level set so that when the norm stops decreasing, 
		# we terminate the game
		norm_tracker.append(LA.norm(y, 2))

		if argVisualize:
			value_rolling_np = value_rolling.get()
			mesh_bundle=implicit_mesh(value_rolling_np, level=0, \
										spacing=tuple(spacing.tolist()), \
										edge_color=None,  face_color=next(colors))
			viz.update_tube(mesh_bundle, time_step, True)

		
			fig = plt.gcf()
			fig.savefig(join(params.savedict.savepath, rf"murmurations_{t_now:0>3.4f}.jpg"), \
				bbox_inches='tight',facecolor='None')


		itr_end.record(); itr_end.synchronize()

		gpu_time_buffer.append(cp.cuda.get_elapsed_time(itr_start, itr_end)/1e3)
		info(f't: {time_step} | GPU time: {gpu_time_buffer[-1]:.4f} | Norm: {np.linalg.norm(y, 2):.2f}')
			
		if len(norm_tracker)>20: # and cp.all(cp.diff(norm_tracker[:-5])<.01):
			logger.info("Terminating the game!")
			break
	end_time = cputime()

	info(f"Avg. local time: {sum(gpu_time_buffer)/len(gpu_time_buffer):.4f} secs")
	info(f"Total Time: {(end_time-start_time):.4f} secs")

	return value_rolling.get()

In [7]:
valRolling = evolve_brats_in_time(1, False, flock1, finite_diff_data1, params1, )

In [8]:
# This (naively) assumes the speed at every point is 1 m/s
def speed_func_ones(flock):
    gridShape = flock.grid.shape
    return np.ones(gridShape)

def speed_func_actual(flock): #TODO: Implement this 
    return

In [9]:
# VIIM Implementation 
#TODO: Considering Jaxifying these 
#TODO: Go from numpy to cp on all of these
#TODO: Get rid of as many loops as I can using cp._ix, figure out how to do that and where I can't 
#TODO: narrow band instead of computing everywhere on the numerical grid (should be quick if we already have epsilon)
#TODO: Finish up calculating the voronoi interface
#TODO: Extension velocities computation in the positive \epsilon level set of the interface
#TODO: Figure out how to test all of these functions 

# First, since we haven't already done so, initialize the unsigned distance function on the grid

def get_BRAT_value(coordinatePair, valRolling):

    return valRolling[coordinatePair]

def check_if_boundary(index, valRolling):

    xDimMax, yDimMax, omegaDimMax = valRolling.shape

    # Suppose we've computed the value functional for every point on a n-d grid. 
    # Consider a point z on this grid
    # If all 2^n adjacent points of z have a negative value functional, z is not on the boundary 
    # This implies that if ANY adjacent points have a positive value functional, z is on the boundary 

    xIndex = index[0]
    yIndex = index[1]
    omegaIndex = index[2]

    for dXIdx in [-1, 1]:
        for dYIdx in [-1, 1]:
            for dOmegaIdx in [-1, 1]:
                adjXIndex = xIndex + dXIdx
                adjYIndex = yIndex + dYIdx
                adjOmegaIndex = omegaIndex + dOmegaIdx

                # Check to see if this is a valid index; if not continue
                if (adjXIndex < 0) or (adjXIndex >= xDimMax):
                    continue

                if (adjYIndex < 0) or (adjYIndex >= yDimMax):
                    continue

                if (adjOmegaIndex < 0) or (adjOmegaIndex >= omegaDimMax):
                    continue

                coordinatePair = (adjXIndex, adjYIndex, adjOmegaIndex)
                adjValue = get_BRAT_value(coordinatePair=coordinatePair, valRolling=valRolling)

                if adjValue >= 0:
                    return True
                
    return False

def index_to_spatial_location(xVals, yVals, omegaVals, index):
    x = xVals[index[0]]
    y = yVals[index[1]]
    omega = omegaVals[index[2]]

    return x, y, omega 

def calculate_unsigned_distance_naive(boundaryIndices, nonBRATIndices, flock):  #TODO: this is really quite expensive since we compute all the levelsets (all distances away from the 0 level set) in this initialization.
                                                                                #      We can probably get away with a narrow band approach, where we only consider the nearest \epsilon points to the level set, and use this 
                                                                                #      to iteratively solve the level set equation. Drops our complexity to O(N^3) to O(kN^3) where k << 1
                                                                                #      also this unsigned everywhere --> figure out somewhat to set things inside the interface as negative 

    # For all points on the grid, get the distance to the nearest interface (aka, 0 level-set)
    udfGrid = np.ones(flock.grid.shape)
    udfGrid[boundaryIndices] = 0 # the unsigned distance function is zero on the boundary 
    xVals = flock.grid.vs[0]
    yVals = flock.grid.vs[1]
    omegaVals = flock.grid.vs[2]
    boundaryCoordinates = []

    for boundaryIndex in boundaryIndices:
        boundaryCoordinate = index_to_spatial_location(xVals, yVals, omegaVals, boundaryIndex)
        boundaryCoordinates.append(boundaryCoordinate)

    boundaryCoordinates = np.asarray(boundaryCoordinates)
    
    for nonBRATIndex in nonBRATIndices:
        nonBRATCoordinate = np.asarray(index_to_spatial_location(xVals, yVals, omegaVals, nonBRATIndex))
        normsAcrossInterface = np.linalg.norm(nonBRATCoordinate - boundaryCoordinate)
        minNorm = np.min(normsAcrossInterface, axis=None)
        udfGrid[nonBRATIndex] = minNorm 
    
    return udfGrid 

def unsigned_distance_function(flock, valRolling):      

    # Get all BRAT indexes: these are the indices of points on the grid where the value is less than 0
    BRATIndices = np.where(valRolling < 0)
    BRATIndices = list(zip(BRATIndices[0], BRATIndices[1], BRATIndices[2]))

    # Get all nonBRAT indexes: these are the indices of points on the grid where the value is greater than or equal to 0
    nonBRATIndices = np.where(valRolling >= 0)
    nonBRATIndices = list(zip(nonBRATIndices[0], nonBRATIndices[1], nonBRATIndices[2]))

    boundaryIndices = []
 
    # First, find all points comprising the interface (aka, the 0-level set) 
    for index in BRATIndices:
        onBoundary = check_if_boundary(index, valRolling)
        if onBoundary:
            boundaryIndices.append(tuple([index[0], index[1], index[2]]))
    
    # On the grid, calculate for each point the distance to the closest point on the interface 
    udfGrid = calculate_unsigned_distance_naive(boundaryIndices, nonBRATIndices, flock)
    
    return udfGrid

# Second, evolve the unsigned distance function a small step \delta t by solving the associated PDE 

def solve_zero_level_set_pde(udfGrid, speed_on_interface, delta_t, spatial_discretization):
    #TODO: assuming that we've defined the speed properly so that it's defined in the normal direction of every point on the interface (aka, inside to out) 
    # This is Eqn. (6.17) in Sethian, 1999 
    # TODO: This requires F to be convex: Once F is actually defined, we can figure out better ways to solve for a small time step dt 
    # TODO: I highly doubt F is convex, so will change this in the very near future 
    # TODO: First order scheme, consider going to the second order scheme? 

    def get_left_deriv(udfGrid, axis):
       
        # This is D^{-x}u in (5.6) from Sethian, 1999
        # For a point ijk on the udfGrid, the left spatial partial derivative on, w.l.o.g., the x axis is given by 
        # (\phi^n_{ijk} - \phi^n_{i-1jk}) / \delta_x
        
        # Add ghost points on the computational domain by extrapolating the udf function 
        extrapolatedUDFGrid = addGhostExtrapolate(udfGrid, dim=axis, width=1)
        axisDiscretization = spatial_discretization[axis]

        # Because we extrapolated indices 1,...,axis_size correlates to 0,...,axis_size - 1 on our UDF
        indicesOne = cp.arange(start=1, stop=size(extrapolatedUDFGrid, axis)-1, dtype=cp.intp)
        indicesTwo = indicesOne - 1

        # Finite difference approximation of the left derivative on the grid
        # TODO: Might want to get an understanding of the error bounds here since we extrapolate on the edge of the grid
        # possibly relevant to disturbance term in HJI calculation (should be bounded by CFL?)
        ijkPhi = cp.take(extrapolatedUDFGrid, indicesOne, axis=axis)
        ijkPhiShiftedLeft = cp.take(extrapolatedUDFGrid, indicesTwo, axis=axis)

        dL = (ijkPhi - ijkPhiShiftedLeft) / axisDiscretization
    
        return dL
    
    def get_right_deriv(udfGrid, axis):
        # This is D^{+x}u in (5.6) from Sethian, 1999
        # For a point ijk on the udfGrid, the right spatial partial derivative on, w.l.o.g., the x axis is given by 
        # (\phi^n_{i+1jk} - \phi^n_{ijk}) / \delta_x
        extrapolatedUDFGrid = addGhostExtrapolate(udfGrid, dim=axis, width=1)
        axisDiscretization = spatial_discretization[axis]

        # Because we extrapolated indices 1,...,axis_size correlates to 0,...,axis_size - 1 on our UDF
        indicesOne = cp.arange(start=1, stop=size(extrapolatedUDFGrid, axis)-1, dtype=cp.intp)
        indicesTwo = indicesOne + 1

        # Finite difference approximation of the right derivative on the grid
        # TODO: Might want to get an understanding of the error bounds here since we extrapolate on the edge of the grid
        # possibly relevant to disturbance term in HJI calculation (should be bounded by CFL?)
        ijkPhi = cp.take(extrapolatedUDFGrid, indicesOne, axis=axis)
        ijkPhiShiftedRight = cp.take(extrapolatedUDFGrid, indicesTwo, axis=axis)

        dR = (ijkPhiShiftedRight - ijkPhi) / axisDiscretization

        return dR
    
    def get_nabla_right(leftDerivs, rightDerivs, nDims):
        nablaRight = cp.zeros(udfGrid.shape)

        for i in range(nDims):
            nablaRight += cp.max(leftDerivs[i], axis=None) + cp.min(rightDerivs[i], axis=None)

        return nablaRight
    
    def get_nabla_left(leftDerivs, rightDerivs, nDims):
        nablaLeft = cp.zeros(udfGrid.shape)

        for i in range(nDims):
            nablaLeft += cp.max(rightDerivs[i], axis=None) + cp.min(leftDerivs[i], axis=None)

        return nablaLeft
    
    nDims = udfGrid.ndims
    lDerivs = [get_left_deriv(udfGrid, i) for i in range(nDims)]
    rDerivs = [get_right_deriv(udfGrid, i) for i in range(nDims)]

    nRight = get_nabla_right(lDerivs, rDerivs)
    nLeft = get_nabla_left(lDerivs, rDerivs)

    udfNew = np.zeros(udfGrid)

    # (6.17) first order, convex speed finite difference scheme from Sethian, 1999
    udfNew = udfGrid - delta_t * (cp.max(speed_on_interface, 0) * nRight + cp.min(speed_on_interface, 0) * nLeft)
    
    return udfNew

# Second, reconstruct the interface after time \delta t by finding the Voronoi interface reconstructed from the \epsilon level-sets

def get_epsilon_brat(udfGrid, epsilon, BRATIndices, flock):
    xVals = flock.grid.vs[0]
    yVals = flock.grid.vs[1]
    omegaVals = flock.grid.vs[2]
    epsilonIndices = {} # instantiating as a set because of possible redundancies in computation 

    for BRATIndex in BRATIndices: # TODO: without looping? probably cp._ix can make this a lot quicker... 
        BRATCoordinate = np.asarray(index_to_spatial_location(xVals, yVals, omegaVals, BRATIndex))
        normsAcrossInterface = np.linalg.norm(BRATCoordinate - udfGrid) # TODO: too naive, consider narrow band 
        #TODO: do i actually need this step? maybe just np.where on \epsilon? 
        epsNorms = normsAcrossInterface - epsilon # if a norm is near epsilon, it's now near zero
        np.round(epsNorms, decimals=0, out=epsNorms) #TODO: decimals depends on magnitude of epsilon, big source of error probably
        indices = np.where(epsNorms == 0)
        indices = list(zip(indices[0], indices[1], indices[2]))
        epsilonIndices.update(indices)
        # put each index in indices in the set 
    return epsilonIndices

def get_voronoi_interface(udfGrid, epsilonIndices):
    # Get all the points equidistant to epsilon_indices
    # TODO: Eikonal solver for this?
    # TODO: finish this!
    for epsilonIndex in epsilonIndices:
        return
    

# Third, update the speed function 
# TODO: Do this (relevant paper: extension velocity paper by Sethian and Saye(?), Chapter 11 in Sethian, 1999)

In [10]:
# Basic debugging 
#TODO: continue debugging this and make it quicker lolz
arr = unsigned_distance_function(flock1, valRolling)

In [30]:
arrExtrapolated = addGhostExtrapolate(arr, dim = 0,width=1)
indices1 = cp.arange(1,size(arrExtrapolated, 0) , dtype=cp.intp)
print(size(arrExtrapolated, 0))
indices2 = indices1 - 1
# derivative along this axis for 
dL = (np.take(arrExtrapolated, indices2, axis=0) - np.take(arrExtrapolated, indices1, axis=0)) / 5
dL.shape

152


(152, 150, 150)

In [12]:
arr
apple = arr.copy()
apple = apple[apple == 0]
apple

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
grid = pde.UnitGrid([20,20])
F = np.array([1,1,1]) #TODO: i can also recreate the F by hand via https://py-pde.readthedocs.io/en/latest/examples_gallery/pde_heterogeneous_diffusion.html
eq = pde.PDE({"phi": "-{F} * | d_dx(phi) |"})